In [1]:
import cv2 as cv
import os

## Saving the reversed video without saving each frame in local disk

In [6]:

# Define the path to the input video
video_path = "VIDEOS/woman_jogging.mp4"

# Open the video capture
cap = cv.VideoCapture(video_path)

if not cap.isOpened():
    raise Exception("Error! couldn't open the video!")

# Define the output video path
output_path = 'VIDEOS/reversed_optimized.mp4'

# Get video properties
w, h, fps = (cap.get(int(x)) for x in (cv.CAP_PROP_FRAME_WIDTH, cv.CAP_PROP_FRAME_HEIGHT, cv.CAP_PROP_FPS))

# Define the codec and create VideoWriter object
fourcc = cv.VideoWriter_fourcc(*'mp4v')
output = cv.VideoWriter(output_path, fourcc, fps, (w, h))

# List to store frames in memory
frames = []

# Read all frames from the video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frames.append(frame)

# Release the video capture
cap.release()

# Reverse the frames list
frames.reverse()

# Write the reversed frames to the output video
for frame in frames:
    output.write(frame)
    cv.imshow('Reverse Video', frame)

    if cv.waitKey(1) & 0xff == ord('p'):
        break

# Release resources
cap.release()
output.release()
cv.destroyAllWindows()


## Defining a function to save any video from web in reverse order

In [ ]:

def save_reversed_video(video_url, save_video=False, filename=None):
    try:
        # Open the video stream from the URL
        cap = cv.VideoCapture(video_url)
        if not cap.isOpened():
            raise ValueError("Couldn't open the video stream from URL")

        # Get video properties
        w = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
        h = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv.CAP_PROP_FPS)

        # Define the codec and create VideoWriter object if saving
        output = None
        if save_video and filename:
            output_path = os.path.join("VIDEOS", filename)
            fourcc = cv.VideoWriter_fourcc(*'mp4v')
            output = cv.VideoWriter(output_path, fourcc, fps, (w, h))

        # Read and store all frames in memory
        frames = []
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frames.append(frame)

        # Release the video capture
        cap.release()

        # Reverse the frames and write/display them
        for frame in reversed(frames):
            if output:
                output.write(frame)
            resized_frame = cv.resize(frame, (int(0.5*frame.shape[1]), int(0.5*frame.shape[0])))
            cv.imshow('Reversed Video', resized_frame)
            if cv.waitKey(1) & 0xFF == ord('p'):
                break

        # Clean up resources
        if output:
            output.release()
        cv.destroyAllWindows()

        if output:
            print(f"Reversed video saved successfully as {filename}")
        else:
            print("Reversed video playback complete.")

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # Ensure resources are released if an error occurs
        if cap.isOpened():
            cap.release()
        if output and output.isOpened():
            output.release()
        cv.destroyAllWindows()

# Example usage
video_url = "https://videos.pexels.com/video-files/6120101/6120101-hd_1920_1080_30fps.mp4"
filename = 'reversed_video.mp4'
save_reversed_video(video_url, save_video=True, filename=filename)


## Stacking both Original and Reversed frames horizontally 

In [21]:

def save_reversed_and_stacked_video(video_url, save_video=False, reversed_filename=None, stacked_filename=None):
    try:
        # Open the video stream from the URL
        cap = cv.VideoCapture(video_url)
        if not cap.isOpened():
            raise ValueError("Couldn't open the video stream from URL")

        # Get video properties
        w, h, fps = (int(cap.get(x)) for x in (cv.CAP_PROP_FRAME_WIDTH, cv.CAP_PROP_FRAME_HEIGHT, cv.CAP_PROP_FPS))
        frame_count = int(cap.get(cv.CAP_PROP_FRAME_COUNT))
        video_duration = frame_count / fps

        # Define the codec and create VideoWriter objects if saving
        reversed_output, stacked_output = None, None
        if save_video:
            if reversed_filename:
                reversed_output_path = os.path.join('VIDEOS', reversed_filename)
                fourcc = cv.VideoWriter_fourcc(*'mp4v')
                reversed_output = cv.VideoWriter(reversed_output_path, fourcc, fps, (w, h))

            if stacked_filename:
                stacked_output_path = os.path.join('VIDEOS', stacked_filename)
                fourcc = cv.VideoWriter_fourcc(*'mp4v')
                stacked_output = cv.VideoWriter(stacked_output_path, fourcc, fps, (2 * w, h))  # Stacked width is doubled

        # Read and store all frames in memory
        frames = []
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frames.append(frame)

        # Release the video capture
        cap.release()

        # Define font for timer display
        font = cv.FONT_HERSHEY_SIMPLEX
        font_scale = 2
        font_color_forward = (255, 0, 0)
        font_color_backward = (0, 0, 255)

        font_thickness = 3
        position_original = (20, 70)  # Top-left corner for original frame timer
        position_reversed = (w + 20, 70)  # Top-left corner for reversed frame timer

        # Process and save each frame with timers
        for i, (original_frame, reversed_frame) in enumerate(zip(frames, reversed(frames))):
            # Calculate forward and backward times in seconds
            forward_time = i / fps
            backward_time = video_duration - forward_time

            # Format the times as MM:SS
            forward_timer_text = f"{int(forward_time // 60):02}:{int(forward_time % 60):02}"
            backward_timer_text = f"{int(backward_time // 60):02}:{int(backward_time % 60):02}"

            if reversed_output:
                reversed_output.write(reversed_frame)  # Save reversed frames

            # Stack original and reversed frames horizontally
            stacked_frame = cv.hconcat([original_frame, reversed_frame])
            # Overlay the timers on the stacked frame
            cv.putText(stacked_frame, forward_timer_text, position_original, font, font_scale, font_color_forward, font_thickness)
            cv.putText(stacked_frame, backward_timer_text, position_reversed, font, font_scale, font_color_backward, font_thickness)

            if stacked_output:
                stacked_output.write(stacked_frame)  # Save stacked frames

            # Show the stacked frame
            stacked_frame_resized = cv.resize(stacked_frame, (int(0.4*stacked_frame.shape[1]), int(0.4*stacked_frame.shape[0])))
            cv.imshow('Original and Reversed (Stacked)', stacked_frame_resized)
            if cv.waitKey(fps) & 0xFF == ord('p'):
                break

        # Clean up resources
        if reversed_output:
            reversed_output.release()
        if stacked_output:
            stacked_output.release()
        cv.destroyAllWindows()

        # Print status
        if reversed_output:
            print(f"Reversed video saved successfully as {reversed_filename}")
        if stacked_output:
            print(f"Stacked video saved successfully as {stacked_filename}")
        else:
            print("Reversed and stacked video playback complete.")

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # Ensure resources are released if an error occurs
        if cap.isOpened():
            cap.release()
        if reversed_output and reversed_output.isOpened():
            reversed_output.release()
        if stacked_output and stacked_output.isOpened():
            stacked_output.release()
        cv.destroyAllWindows()



In [ ]:
# Example usage
video_url = "https://videos.pexels.com/video-files/6120101/6120101-hd_1920_1080_30fps.mp4"
reversed_filename = 'reversed_video.mp4'
stacked_filename = 'stacked_video_with_timers.mp4'
save_reversed_and_stacked_video(video_url, save_video=True, reversed_filename=reversed_filename, 
                                           stacked_filename=stacked_filename)

In [ ]:
save_reversed_and_stacked_video("https://videos.pexels.com/video-files/7187055/7187055-hd_1920_1080_24fps.mp4", 
                                           save_video=True, reversed_filename="reversed_video_1.mp4", 
                                           stacked_filename="stacked_video_1.mp4")

In [ ]:
save_reversed_and_stacked_video("https://videos.pexels.com/video-files/5246412/5246412-hd_1920_1080_30fps.mp4",
                                           save_video=True, reversed_filename="reversed_video_2.mp4", 
                                           stacked_filename="stacked_video_2.mp4")

In [ ]:
save_reversed_and_stacked_video("https://videos.pexels.com/video-files/3815194/3815194-hd_1920_1080_30fps.mp4")

In [ ]:
save_reversed_and_stacked_video("https://videos.pexels.com/video-files/5803633/5803633-uhd_2560_1440_25fps.mp4", 
                                           save_video=True, reversed_filename="reversed_video_3.mp4", 
                                           stacked_filename="stacked_video_3.mp4")

In [ ]:
save_reversed_and_stacked_video("https://videos.pexels.com/video-files/6537653/6537653-hd_1920_1080_24fps.mp4",
                                           save_video=True, reversed_filename="reversed_video_4.mp4", 
                                           stacked_filename="stacked_video_4.mp4")